To demonstrate how to use Pearson correlation algorithm in Neo4j we will use the data from [“Young People Survey Kaggle”](https://www.kaggle.com/miroslavsabo/young-people-survey/home) dataset made available by Miroslav Sabo. It contains results of 1010 filled out surveys with questions ranging from music preferences, hobbies & interests to phobias.

为了演示如何在Neo4j中使用Pearson相关算法，我们将使用Miroslav Sabo提供的青少年调查Kaggle数据集。它包含了1010项调查的结果，这些调查的问题包括音乐喜好、爱好和利益恐惧症。

The nice thing about using Pearson correlation in scoring scenarios is that it takes into account when voters are generally more inclined to give higher or lower scores as it compares each score to the average score of the user.

在评分场景中使用皮尔逊相关性的好处是，当将每个分数与用户的平均分数进行比较时，它考虑到选民通常更倾向于给出更高或更低的分数。

# Import

Download the dataset and copy it into $Neo4j/import folder. Each row in the responses.csv file represents a single survey with 150 questions filled out. We store it in Neo4j as a single node.

下载数据集并将其复制到$Neo4j/import文件夹中。csv文件中的每一行表示一个包含150个问题的调查。我们将它作为单个节点存储在Neo4j中。

[APOC安装](https://blog.csdn.net/timefisher/article/details/79893826)

# Preprocessing

Most of the answers range from one to five where five is defined as Strongly agree and one as Strongly disagree . They appear as strings in the csv file and we have to convert them to integers first.

大多数答案的范围从1到5，其中5被定义为完全同意，1被定义为完全不同意。它们作为字符串出现在csv文件中，我们必须首先将它们转换为整数。

# Category properties

Some of the answers are categorical. An example is the alcohol question, where possible answers are never , social drinker and drink a lot . As we would like to convert some of them to vectors let s examine all the possible answers they have.

有些答案是绝对的。举个例子，酒精问题，可能的答案是永远不会，社交饮酒者和大量饮酒。因为我们想把它们中的一些转换成向量让我们检查一下它们所有可能的答案。

Let s vectorize gender, internet and alcohol answers. We will scale them between one to five to match the integer answers range.

让我们将性别、网络和酒精答案量化。我们将在1到5之间缩放它们，以匹配整数答案范围。

## Gender encoding

## Internet encoding

## Alcohol encoding

# Dimensionality reduction

There are 150 answers in our dataset that we could use as features. This is a great opportunity to perform some basic dimensionality reduction of the features.

在我们的数据集中有150个答案，我们可以用它们作为特性。这是一个很好的机会来执行一些基本的降维功能。

I came across an article about [dimensionality reduction techniques](https://www.analyticsvidhya.com/blog/2018/08/dimensionality-reduction-techniques-python/) written by [Pulkit Sharma](https://www.analyticsvidhya.com/blog/author/pulkits/). It describes twelve dimensionality reduction techniques, and in this post, we will use the first two, which are the **low variance filter** and the **high correlation filter**.

我偶然看到了一篇由Pulkit Sharma写的关于降维技术的文章。它描述了12种降维技术，在这篇文章中，我们将使用前两种，即低方差滤波器和高相关滤波器。

## Low variance filter

Consider a variable in our dataset where all the observations have the same value, say 1. If we use this variable, do you think it can improve the model we will build? The answer is no, because this variable will have zero variance.

假设我们的数据集中有一个变量，其中所有的观察值都是相同的，比如1。如果我们使用这个变量，你认为它能改进我们将要建立的模型吗?答案是否定的，因为这个变量的方差为零。

[source](https://www.analyticsvidhya.com/blog/2018/08/dimensionality-reduction-techniques-python/)

We will use the standard deviation metric, which is just the square root of the variance.

我们将使用标准差度量，也就是方差的平方根。

result
![80.png](./picture/80.png)

We can observe that everybody likes to listen to music, watch movies and have fun with friends.

我们可以观察到，每个人都喜欢听音乐，看电影，和朋友一起玩。

Due to the low variance, we will eliminate the following questions from our further analysis:

“Personality”
“Music”
“Dreams”
“Movies”
“Fun with friends”
“Comedy”

## High correlation filter

High correlation between two variables means they have similar trends and are likely to carry similar information. This can bring down the performance of some models drastically (linear and logistic regression models, for instance).

两个变量之间的高相关性意味着它们具有相似的趋势，并且可能携带相似的信息。这可能会大大降低某些模型的性能(例如，线性和逻辑回归模型)。

[source](https://www.analyticsvidhya.com/blog/2018/08/dimensionality-reduction-techniques-python/)

We will use the Pearson correlation coefficient for this task. Pearson correlation adjusts for different location and scale of features, so any kind of linear scaling (normalization) is unnecessary.

我们将使用Pearson相关系数来完成这项任务。皮尔逊相关可以根据特征的不同位置和尺度进行调整，因此没有必要进行任何形式的线性尺度(归一化)。

**Find top 10 correlations for gender feature.**

![81.png](./picture/81.png)

Most correlated feature to gender is weight, which makes sense. The list includes some other stereotypical gender differences like the preference for cars, action, and PC.

与性别最相关的特征是体重，这是有道理的。这份清单还包括其他一些典型的性别差异，比如对汽车、动作和个人电脑的偏好。

Let s now calculate the Pearson correlation between all the features.

现在让我们计算所有特征之间的Pearson相关性。

result ![82.png](./picture/82.png)

Results show nothing surprising. The only one I found interesting was the correlation between snakes and rats.

结果并不令人惊讶。我发现唯一有趣的是蛇和老鼠之间的关系。

We will exclude the following questions due to high correlation from further analysis: 
    
    Medicine Chemistry Shopping centres Physics Opera Animated

# Pearson similarity algorithm

Now that we have completed the preprocessing step we will infer a similarity network between nodes based on the Pearson correlation of the features(answers) of nodes that we haven t excluded.

现在我们已经完成了预处理步骤，我们将根据尚未排除的节点的特征(答案)的Pearson相关性来推断节点之间的相似网络。

In this step we need all the features we will use in our analysis to be normalized between one and five as now, we will fit all the features of the node in a single vector and calculate correlations between them.

在这一步中，我们需要将分析中使用的所有特征规范化为1到5，就像现在一样，我们将把节点的所有特征放入一个向量中，并计算它们之间的相关性。

## Min-max normalization

Three of the features are not normalized between one to five. These are 

    Height 
    Number of siblings 
    Weight 

Normalize height property between one to five. We won't use the other two.

其中三个特性没有在1到5之间进行标准化。这些是

    身高
    数
    体重

正常化身高属性在1到5之间。我们不用另外两个。

## Similarity network

We grab all the features and infer the similarity network. We always want to use similarityCutoff parameter and optionally topK parameter to prevent ending up with a complete graph, where all nodes are connected between each other. Here we use similarityCutoff: 0.75 and topK: 5. Find more information in the documentation.
        
        我们抓住所有的特征并推断出相似网络。我们总是希望使用similarityCutoff参数和可选的topK参数来避免最终得到一个完整的图，其中所有节点都彼此连接。这里我们使用similarityCutoff: 0.75和topK: 5。在文档中找到更多信息。

# Community detection

Now that we have inferred a similarity network in our graph, we will try to find communities of similar persons with the help of [Louvain algorithm](https://neo4j.com/docs/graph-algorithms/current/algorithms/louvain/).


## Apoc.group.nodes

For a quick overview of community detection results in Neo4j Browser, we can use apoc.group.nodes. We define the labels we want to include and group by a certain property. In the config part, we define which aggregations we want to perform and get returned in the visualization. Find more in the [documentation](https://neo4j-contrib.github.io/neo4j-apoc-procedures/#_node_grouping).

为了快速概述Neo4j浏览器中的社区检测结果，我们可以使用.group.nodes。我们定义要包含的标签，并根据某个属性进行分组。在配置部分中，我们定义要执行哪些聚合并在可视化中返回。在文档中可以找到更多信息。

Results![83.png](./picture/83.png)

# Community preferences

To get to know our communities better, we will examine their average top and bottom 3 preferences.

为了更好地了解我们的社区，我们将检查他们的平均顶部和底部3个偏好。

Results ![84.png](./picture/84.png)

Results are quite interesting. Just looking at the male percentage it is safe to say that the communities are almost all based on gender. 
    
    结果非常有趣。只要看看男性的比例，就可以肯定地说，这些社区几乎都是基于性别的。

 The biggest community are 220 ladies, who strongly agree with Compassion to animals , Romantic and interestingly Borrowed stuff but disagree with Metal , Western and Writing . Second biggest community, mostly male, agree with Cheating in school , Action and PC . They also don t agree with Writing . Makes sense as the survey was filled out by students from Slovakia.

    最大的社区有220位女士，她们强烈赞同对动物的同情，浪漫和有趣的借来的东西，但不同意金属、西方和写作。第二大社区，大多数是男性，同意在学校、Action和PC上作弊。他们也不同意写作。这项调查是由斯洛伐克的学生填写的。

# Gephi visualization

Let s finish off with a nice visualization of our communities in [Gephi](https://gephi.org/). You need to have the streaming plugin enabled in Gephi and then we can export the graph from Neo4j using the APOC procedure apoc.gephi.add.

最后让我们来看看我们在Gephi的社区。您需要在Gephi中启用流媒体插件，然后我们可以使用APOC过程. Gephi .add从Neo4j导出图形。

After a bit of tweaking in Gephi, I came up with this visualization. Similarly as with apoc.group.nodes visualization we can observe, that the biggest communities are quite connected between each other.

在对Gephi做了一些调整之后，我想到了这个可视化。与.group类似。节点可视化我们可以观察到，最大的社区之间是相互连接的。